In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3

import numpy as np
from matplotlib import pyplot as plt
import time

In [ ]:
class AKPD(object):

    def __init__(self, aws_credentials):
        self.client = boto3.client(
            "sagemaker-runtime", 
            region_name="eu-west-1", 
            aws_access_key_id=aws_credentials['aws_access_key_id'], 
            aws_secret_access_key=aws_credentials['aws_secret_access_key']
        
        )

    def predict_keypoints(self, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, camera_metadata):
        body = [{
            'leftCropUrl': left_crop_url,
            'rightCropUrl': right_crop_url,
            'leftCropMetadata': left_crop_metadata,
            'rightCropMetadata': right_crop_metadata,
            'cameraMetadata': camera_metadata,
            'id': 1
        }]

        body_str = json.dumps(body).replace("'", '"')

        resp = self.client.invoke_endpoint(EndpointName='auto-keypoints', ContentType='application/json', Body=body_str)
        akpd_keypoints_str = resp['Body'].read()
        akpd_keypoints = json.loads(akpd_keypoints_str.decode("utf-8"))
        return akpd_keypoints

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    select * from keypoint_annotations where pen_id=61 and captured_at between '2019-09-13' and '2019-09-21'
    and keypoints -> 'leftCrop' is not null
    and keypoints -> 'rightCrop' is not null;
"""

df = rds_access_utils.extract_from_database(query)
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd = AKPD(aws_credentials)

to_tensor_transform = ToTensor()

# initialize data transforms so that we can run inference with biomass neural network
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')
akpd_scorer_network = load_model('/root/data/alok/biomass_estimation/playground/akpd_scorer_model_TF.h5') # make this better

<h1> Function to generate weight prediction and confidence score </h1>

In [ ]:
def generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, keypoints, cm):
    
    akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, cm)

    # run AKPD scoring network
    input_sample = {
        'keypoints': akpd_keypoints[0],
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    akpd_score = generate_confidence_score(input_sample, akpd_scorer_network)

    # run biomass estimation
    input_sample = {
        'keypoints': akpd_keypoints[0],
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    akpd_weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    
    # run manual scoring network
    input_sample = {
        'keypoints': keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    manual_score = generate_confidence_score(input_sample, akpd_scorer_network)

    # run biomass estimation
    input_sample = {
        'keypoints': keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    manual_weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    
    print(akpd_score, akpd_weight_prediction, manual_score, manual_weight_prediction)
    return akpd_keypoints, akpd_score, akpd_weight_prediction, manual_score, manual_weight_prediction


In [ ]:
weight_score_dict = {}

args = []
for idx, row in df.head(1000).iterrows():
    left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata,
    cm = row.camera_metadata
    keypoints = row.keypoints
    row_id = idx
    akpd_keypoints, akpd_score, akpd_weight_prediction, manual_score, manual_weight_prediction = generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, keypoints, cm)
    weight_score_dict[row_id] = {
        'akpd_keypoints': akpd_keypoints,
        'akpd_score': akpd_score,
        'akpd_weight_prediction': akpd_weight_prediction,
        'manual_score': manual_score,
        'manual_weight_prediction': manual_weight_prediction
    }
    


In [ ]:
df['akpd_weight'], df['akpd_score'], df['manual_weight'], df['manual_score'] = np.nan, np.nan, np.nan, np.nan
for idx, row in df.iterrows():
    if idx in weight_score_dict.keys():
        df.at[idx, 'akpd_weight'] = weight_score_dict[idx]['akpd_weight_prediction']
        df.at[idx, 'akpd_score'] = weight_score_dict[idx]['akpd_score']
        df.at[idx, 'manual_weight'] = weight_score_dict[idx]['manual_weight_prediction']
        df.at[idx, 'manual_score'] = weight_score_dict[idx]['manual_score']

In [ ]:
score_mask = df.akpd_score > 0.9
df[score_mask].akpd_weight.mean(), df[score_mask].manual_weight.mean()

In [ ]:
plt.figure(figsize=(20, 10))
mask = df.score > 0.8
plt.hist(df[mask].weight, bins=10)
plt.xlabel('Predicted weight (grams)')
plt.ylabel('Frequency')
plt.title('Predicted weight distribution for IMR - 11/09-11/13')
plt.grid()
plt.show()

In [ ]:
df[mask].weight.mean()

In [ ]:
def display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, side='both', overlay_keypoints=True, show_labels=False):
    assert side == 'left' or side == 'right' or side == 'both', \
        'Invalid side value: {}'.format(side)

    if side == 'left' or side == 'right':
        fig, ax = plt.subplots(figsize=(20, 10))
        image_f = left_image_f if side == 'left' else right_image_f
        keypoints = left_keypoints if side == 'left' else right_keypoints
        image = plt.imread(image_f)
        ax.imshow(image)

        if overlay_keypoints:
            for bp, kp in keypoints.items():
#                 ax.scatter([kp[0]], [kp[1]], color='red', s=1)
                ax.scatter([kp[0]], [kp[1]], color='red', s=200, alpha=0.3)
                if show_labels:
                    ax.annotate(bp, (kp[0], kp[1]), color='red')
    else:
        fig, axes = plt.subplots(2, 1, figsize=(20, 20))
        left_image = plt.imread(left_image_f)
        right_image = plt.imread(right_image_f)
        axes[0].imshow(left_image)
        axes[1].imshow(right_image)
        if overlay_keypoints:
            for bp, kp in left_keypoints.items():
#                 axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
                axes[0].scatter([kp[0]], [kp[1]], color='red', s=200, alpha=0.3)
                if show_labels:
                    axes[0].annotate(bp, (kp[0], kp[1]), color='red')
            for bp, kp in right_keypoints.items():
#                 axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
                axes[1].scatter([kp[0]], [kp[1]], color='red', s=200, alpha=0.3)
                if show_labels:
                    axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
idx = 24
row = df[mask].sort_values('weight', ascending=False).iloc[idx]

left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
left_image_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
right_image_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata,
cm = row.camera_metadata
row_id = idx

# run AKPD
akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata)
left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in akpd_keypoints[0]['leftCrop']}
right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in akpd_keypoints[0]['rightCrop']}


# run AKPD scoring network
input_sample = {
    'keypoints': akpd_keypoints[0],
    'cm': cm,
    'stereo_pair_id': row_id,
    'single_point_inference': True
}
nomralized_centered_2D_kps = \
    normalize_centered_2D_transform_akpd.__call__(input_sample)

akpd_normalized_kps = akpd_normalization_transform.__call__(nomralized_centered_2D_kps)
tensorized_kps = to_tensor_transform.__call__(akpd_normalized_kps)
score = akpd_scorer_network(tensorized_kps['kp_input']).item()
display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, show_labels=True)


In [ ]:
left_crop_url

In [ ]:
3250 = 2070 * (1+r)**60

In [ ]:
((3250/2070)**(1/63.0)) - 1

In [ ]:
2070*(1.00718)**(51)